In [18]:
import numpy as np
import pandas as pd

path_to_csv = "../data/mortgage_transformed.csv"
df = pd.read_csv(path_to_csv)
df.set_index(df.pop("Unnamed: 0"), inplace=True)

In [14]:
df.columns

Index(['id', 'time', 'orig_time', 'first_time', 'mat_time', 'balance_time',
       'LTV_time', 'interest_rate_time', 'hpi_time', 'gdp_time', 'uer_time',
       'REtype_CO_orig_time', 'REtype_PU_orig_time', 'REtype_SF_orig_time',
       'investor_orig_time', 'balance_orig_time', 'FICO_orig_time',
       'LTV_orig_time', 'Interest_Rate_orig_time', 'hpi_orig_time',
       'default_time', 'payoff_time', 'status_time', 'sample', 'result'],
      dtype='object')

# Creating the model

Let us select the columns, that are significant when it comes to making predictions.

In [15]:
df.head()

,id,time,orig_time,first_time,mat_time,balance_time,LTV_time,interest_rate_time,hpi_time,gdp_time,...,balance_orig_time,FICO_orig_time,LTV_orig_time,Interest_Rate_orig_time,hpi_orig_time,default_time,payoff_time,status_time,sample,result
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0,1,32,-7,25,113,35877.03,25.872559,9.200,186.12,1.104163,...,45000.0,715,69.4,9.2,87.03,0.0,0.0,0.0,public,0.0
1,1,40,-7,25,113,32388.30,27.828850,9.200,156.21,1.585966,...,45000.0,715,69.4,9.2,87.03,0.0,0.0,0.0,public,1.0
2,4,28,-2,25,119,60576.14,34.672545,10.875,219.67,1.229172,...,63750.0,587,81.8,10.5,97.99,0.0,0.0,0.0,public,0.0
3,4,42,-2,25,119,57950.27,45.256864,9.250,161.00,3.029499,...,63750.0,587,81.8,10.5,97.99,0.0,0.0,0.0,public,0.0
4,4,52,-2,25,119,55332.84,43.177891,10.500,161.13,1.081049,...,63750.0,587,81.8,10.5,97.99,0.0,0.0,0.0,public,0.0


In [19]:
X = df.dropna(axis=0)
y = X.pop("result")

In [22]:
from sklearn.model_selection import train_test_split

columns = ["LTV_time", "balance_time"]
X_train, X_valid, y_train, y_valid = train_test_split(
    X[columns], y,
    train_size=0.7,
    test_size=0.3)

In [23]:
# train a simple model
from xgboost import XGBClassifier

model = XGBClassifier(
            n_estimators=100,
            learning_rate=0.1
        )

model.fit(X_train, y_train)
predictions = model.predict(X_valid)

C:\Users\vojte\Documents\GitHub\mortgage-default-prediction\venv\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
C:\Users\vojte\Documents\GitHub\mortgage-default-prediction\venv\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\vojte\Documents\GitHub\mortgage-default-prediction\venv\lib\site-packages\xgboost\data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. 

[14:01:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [26]:
# now we score the predictions
from sklearn.metrics import accuracy_score

print(f"Our accuracy is {accuracy_score(predictions, y_valid)}")

Our accuracy is 0.5480546676953598


In [ ]:
path_to_model = "../models/first_mod"